# Sensitivity analysis.ipynb


# Data

In [ ]:
import data_preparation
import custom_func
import model, model2
import model_analysis

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# Import data
spill_data = pd.read_excel('Inputs/data_oil_spill_resource_allocation_Arctic_2023.xlsx', sheet_name='spills', header=0).copy()
spill_data = spill_data.sample(n=1000, random_state=42, replace=True)  #+++
current_station_data = pd.read_excel('Inputs/data_oil_spill_resource_allocation_Arctic_2023.xlsx', sheet_name='current', header=0).copy()
current_input_param = pd.read_excel('Inputs/data_oil_spill_resource_allocation_Arctic_2023.xlsx', sheet_name='current input param', header=0).copy()


potential_station_data = pd.read_excel('Inputs/data_oil_spill_resource_allocation_Arctic_2023.xlsx', sheet_name='stations', header=0).copy()
potential_station_data = potential_station_data.iloc[[0, 4,7, 10, 11, 18]]
input_parameters = pd.read_excel('Inputs/data_oil_spill_resource_allocation_Arctic_2023.xlsx', sheet_name='Estimated parameters', header=0).copy()
sensitivity_dataR = gpd.read_file('Inputs/ArcGIS_data/Sensitivity_data/Sensitivity_data5.shp').copy()

In [ ]:
coordinates_st = custom_func.extract_station_coordinate(current_station_data)
current_st_coord = pd.DataFrame(coordinates_st[1]).T.reset_index().set_index('index')
current_st_coord.columns = ['St_Latitude', 'St_Longitude']
display(current_st_coord)

coordinates_st2 = custom_func.extract_station_coordinate(potential_station_data)
potential_st_coord = pd.DataFrame(coordinates_st2[1]).T.reset_index().set_index('index')
potential_st_coord.columns = ['St_Latitude', 'St_Longitude']
display(potential_st_coord)

coordinates_spill = custom_func.extract_spill_coordinate(spill_data)
spill_coord = pd.DataFrame(coordinates_spill[1]).T.reset_index().set_index('index')
spill_coord.columns = ['St_Latitude', 'St_Longitude']
display(spill_coord)

# Model Configuration 3 (Current facility location)


In [ ]:
current_vs_proposed = 'current'
NumberStMax = 6  # Do I need this for current setup?? ++
DistanceMax = 15
TimeRMax = 220

In [ ]:
Stations, OilSpills, ResourcesD, Demand, Availability, Eff, Distance, TimeR, Cf_s, Cu_sor \
= data_preparation.generate_input_data(current_station_data, spill_data, current_input_param)


#max(Distance.values())
SizeSpill = SizeSpill_R = list(spill_data['Spill size']).copy()
Sensitivity_R = custom_func.calculate_sensitivity(coordinates_spill[0], sensitivity_dataR)
# normalize
SizeSpill_n = dict(zip(OilSpills, custom_func.normalize(SizeSpill_R, min(SizeSpill_R), max(SizeSpill_R))))
Sensitivity_n = dict(zip(OilSpills, custom_func.normalize(Sensitivity_R, min(Sensitivity_R), max(Sensitivity_R))))

Distance_n = dict(zip(Distance.keys(), custom_func.normalize(Distance.values(), min(Distance.values()), max(Distance.values()))))
# (x_0-x_min)/(x_max-x_min)
Effectiveness_n = dict(zip(Eff.keys(), custom_func.normalize(Eff.values(), min(Eff.values()), max(Eff.values()))))

In [ ]:
# Modeling based on current setup
model, select, deploy, mvars, names, values, \
        spill_df, station_df, select_1s, deploy_1s, ResponseTimeM, coverage_percentage, assignment  \
    = model2.solve(Stations, OilSpills, ResourcesD, coordinates_st, coordinates_spill, SizeSpill, SizeSpill_n, \
          Demand, Sensitivity_R, Sensitivity_n, Eff, Effectiveness_n, Availability, NumberStMax, Distance, Distance_n, DistanceMax,  Cf_s, Cu_sor)

## Fig8(a) Network diagram of current facility

In [ ]:
model_analysis.draw_network_diagram(DistanceMax, NumberStMax, Sensitivity_R, spill_df, station_df, ResponseTimeM, coverage_percentage,
                         assignment, deploy_1s, select_1s, current_vs_proposed)

In [ ]:
assignment_line = pd.DataFrame(columns=list(assignment.columns.values))
for i, o in enumerate(assignment['Spill #'].unique()):
    minD = min(assignment.loc[assignment['Spill #'] == o]['Distance'])
    df = assignment.iloc[np.where(assignment['Distance'] == minD)[0]]
    assignment_line = pd.concat([assignment_line, df])

resource_allocation_c = assignment_line[['Station no.', 'Resource Type', 'Quantity deployed']]
resource_allocation_c = resource_allocation_c.sort_values(by=['Station no.'])
resource_allocation_c = resource_allocation_c.groupby(['Station no.', 'Resource Type'])['Quantity deployed'].mean().reset_index()
resource_allocation_c
resource_allocation_c['Facility'] = 'Current'

# Model Configuration 4 (Proposed 5 facility location)
Resource allocation

In [ ]:
NumberStMax = 5
current_vs_proposed = 'proposed'

In [ ]:
# Input param
Stations, OilSpills, ResourcesD, Demand, Availability, Eff, Distance, TimeR, Cf_s, Cu_sor \
= data_preparation.generate_input_data(potential_station_data, spill_data, input_parameters)

SizeSpill = SizeSpill_R = list(spill_data['Spill size']).copy()
Sensitivity_R = custom_func.calculate_sensitivity(coordinates_spill[0], sensitivity_dataR)
# normalize
SizeSpill_n = dict(zip(OilSpills, custom_func.normalize(SizeSpill_R, min(SizeSpill_R), max(SizeSpill_R))))
Sensitivity_n = dict(zip(OilSpills, custom_func.normalize(Sensitivity_R, min(Sensitivity_R), max(Sensitivity_R))))
Distance_n = dict(zip(Distance.keys(), custom_func.normalize(Distance.values(), min(Distance.values()), max(Distance.values()))))
# (x_0-x_min)/(x_max-x_min)
Effectiveness_n = dict(zip(Eff.keys(), custom_func.normalize(Eff.values(), min(Eff.values()), max(Eff.values()))))

# Model
model, select, deploy, mvars, names, values, \
        spill_df, station_df, select_1s, deploy_1s, ResponseTimeM, coverage_percentage, assignment  \
    = model2.solve(Stations, OilSpills, ResourcesD, coordinates_st2, coordinates_spill, SizeSpill, SizeSpill_n, \
          Demand, Sensitivity_R, Sensitivity_n, Eff, Effectiveness_n, Availability, NumberStMax, Distance, Distance_n, DistanceMax,  Cf_s, Cu_sor)

## Fig8(b) Network diagram of proposed 5 facility

In [ ]:
# Draw the network
assignment_line = model_analysis.draw_network_diagram(DistanceMax, NumberStMax, Sensitivity_R, spill_df, station_df, ResponseTimeM, coverage_percentage,
                         assignment, deploy_1s, select_1s, current_vs_proposed)

## Fig7a barplot - quantity stockplie

In [ ]:
assignment_line = pd.DataFrame(columns=list(assignment.columns.values))
for i, o in enumerate(assignment['Spill #'].unique()):
    minD = min(assignment.loc[assignment['Spill #'] == o]['Distance'])
    df = assignment.iloc[np.where(assignment['Distance'] == minD)[0]]
    assignment_line = pd.concat([assignment_line, df])

resource_allocation_p = assignment_line[['Station no.', 'Resource Type', 'Quantity deployed']]
resource_allocation_p = resource_allocation_p.sort_values(by=['Station no.'])
resource_allocation_p = resource_allocation_p.groupby(['Station no.', 'Resource Type'])['Quantity deployed'].mean().reset_index()
resource_allocation_p
resource_allocation_p['Facility'] = 'Proposed'

In [ ]:
resource_allocation = pd.concat([resource_allocation_c, resource_allocation_p])
resource_allocation

In [ ]:
plt.bar(data=resource_allocation,
        x="Station no.", height="Quantity deployed",
        alpha = 0.5, hatch = 'Resource Type'
        ) # color='Facility'

In [ ]:
fig = plt.figure()
ax = sns.barplot(data=resource_allocation,
                 x="Station no.", y="Quantity deployed", errorbar=None,
                 hue='Facility', palette=["g", "yellow"],
                 alpha=0.7)
#sns.set_style(style='white')
ax.grid(False)
plt.show() #, ls='Resource Type'
fig.savefig('Outputs/barplot_compare.png', transparent=True)

In [ ]:
fig10c = plt.subplots(figsize=(7,3.5))
ax = sns.barplot(data=resource_allocation,
                 x="Station no.", y="Quantity deployed", errorbar=None,
                 hue='Facility', palette=["g", "yellow"],
                 alpha=0.7)
sns.set_style(style='white')
ax.grid(False)
plt.savefig('Outputs/barplot_compareHatch.png', transparent=True)

In [ ]:
fig10c = plt.subplots(figsize=(7,3.5))
ax = sns.barplot(data=resource_allocation,
                 x="Station no.", y="Quantity deployed", errorbar=None,
                 hue='Facility', palette=["g", "yellow"],
                 alpha=0.7)
hatches = ['', '', '', '', '', 'o','o', 'o','','','', '', '', '', '', 'o','o', 'o']
# Loop over the bars
for i,thisbar in enumerate(ax.patches):
    # Set a different hatch for each bar
    thisbar.set_hatch(hatches[i])
sns.set_style(style='white')
ax.grid(False)
plt.savefig('Outputs/barplot_compareHatch.png', transparent=True)

In [ ]:
ax.patches

## Fig7b Boxplot - compare coverage, cost, time for current vs proposed

In [ ]:
data_compare = pd.read_excel('Outputs/current vs proposed.xlsx', skiprows=[0])

data_compare.info()
coverage_df_stacked = data_compare[['Current','Proposed']].copy().stack().reset_index()
coverage_df_stacked.columns = ['Index','Facility','Coverage']
cost_df_stacked = data_compare[['Current.1','Proposed.1']].copy().stack().reset_index()
cost_df_stacked.columns = ['Index','Facility','Cost']

time_df_stacked = data_compare[['Current.2','Proposed.2']].copy().stack().reset_index()
time_df_stacked.columns = ['Index','Facility','Response Time']


In [ ]:
def draw_boxplot(data, y, ax):
    sns.violinplot(data= data,x='Facility', y=y, alpha = 0.5)
    sns.stripplot(data= data, x='Facility', y=y)
    plt.ylabel = y


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
draw_boxplot(coverage_df_stacked, 'Coverage', ax1)
draw_boxplot(cost_df_stacked, 'Cost', ax2)
draw_boxplot(time_df_stacked, 'Response Time', ax3)

plt.show()
fig.savefig('Outputs/boxplot_{}.png', transparent=True)

In [ ]:
plt.rcParams["figure.figsize"] = [7.00, 3]
plt.rcParams['axes.edgecolor'] = 'lightgray'
fig, (ax1, ax2, ax3) = plt.subplots(1,3)

sns.boxplot(data= coverage_df_stacked, ax=ax1, x='Facility', y='Coverage',
            boxprops=dict(alpha=.3), palette=["g", "yellow"])
sns.stripplot(data= coverage_df_stacked, ax=ax1, x='Facility', y='Coverage',
              alpha=.3)


sns.boxplot(data= cost_df_stacked, ax=ax2, x='Facility', y='Cost',
            boxprops=dict(alpha=.3), palette=["g", "yellow"])
sns.stripplot(data= cost_df_stacked, ax=ax2, x='Facility', y='Cost',
              alpha=.3)
sns.boxplot(data= time_df_stacked, ax=ax3, x='Facility', y='Response Time',
            boxprops=dict(alpha=.3), palette=["g", "yellow"])
sns.stripplot(data= time_df_stacked, ax=ax3, x='Facility', y='Response Time',
              alpha=.3)
plt.tight_layout()

#plt.axis('off')
# plt.xlabel('')
plt.show()
fig.savefig('Outputs/boxplot.png', transparent=True)

# Model Configuration 5 (Proposed 6 facility location)

In [ ]:
NumberStMax = 6

## Fig8(b) Network diagram of proposed 5 facility

# Multiscene

## grbtool

In [ ]:
# grblogtool
# !pip install grblogtools
import grblogtools as glt
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:

results = glt.parse(["Outputs/Logfiles/model_moo*.log"])
summary = results.summary()
summary.columns

summary = summary[['Runtime','ObjVal', 'ObjBound', 'RelaxObj', 'IterCount']].dropna()
glt.plot(summary.sort_values(by="Runtime"),
         type="bar",
         color="Log")

progress_iteration = results.progress("nodelog")
display(progress_iteration.columns)
progress_iteration[['LogFilePath', 'Incumbent', 'BestBd', 'Gap']]

results.progress("rootlp")

glt.plot(summary.sort_values(by="Runtime"), #
         x = "IterCount", y = "ObjVal", type="scatter",
         color="LogFilePath",
         log_x=True,
         title="Gurobi Performance")

glt.plot(timelines[timelines["Log"]=="model_moo(Mar 01,2217)"], type='line', y='Gap', color='Seed')
glt.plot(summary, x= 'Time', y='Gap', color='Seed', type="line")